In [9]:
%cd ..
%pwd
# %cd code
# should be /code
# 
%load_ext autoreload
%autoreload 2

/home/oli/Research/Joe/agent-goals/code
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from pgmpy.models import BayesianNetwork,  FactorGraph

from pgmpy.readwrite import BIFReader
# reader = BIFReader('data/alarm.bif')
reader = BIFReader('data/asia.bif')
#reader.get_edges()
alarm = reader.get_model()

In [11]:
from pdg import PDG
from pdg.rv import Variable as Var
from pdg.dist import CPT, RawJointDist as RJD
from pdg.alg import interior_pt as ip

In [12]:
alarm_PDG = PDG.from_BN(alarm)

In [15]:
# VERY DANGEROUS!!
#ip.cvx_opt_clusters(alarm_PDG)
#

Cs = list(alarm_PDG.to_markov_net().to_junction_tree().nodes())
print(list(Cs))
print(sorted([len(c) for c in Cs], reverse=True))


import numpy as np
print([np.prod([len(alarm_PDG.vars[X]) for X in C]) for C in Cs])

# prob = ip.cvx_opt_clusters(alarm_PDG, Cs, also_idef=False, dry_run=True, verbose=True)

# print(ns, sum(ns))

[('dysp', 'bronc', 'either'), ('xray', 'tub', 'lung', 'bronc', 'either'), ('xray', 'tub', 'lung', 'bronc', 'smoke'), ('xray', 'tub', 'asia', 'smoke')]
[5, 5, 4, 3]
[8, 32, 32, 16]


In [111]:
smoke,lung,bronc = alarm_PDG.vars['smoke'], alarm_PDG.vars['lung'], alarm_PDG.vars['bronc']
dysp,either = alarm_PDG.vars['dysp'], alarm_PDG.vars['either']

alarm_PDG.factor_product()[smoke,lung,bronc]

,"((yes, yes), yes)","((yes, yes), no)","((yes, no), yes)","((yes, no), no)","((no, yes), yes)","((no, yes), no)","((no, no), yes)","((no, no), no)"
⋆,0.03,0.02,0.27,0.18,0.0015,0.0035,0.1485,0.3465


In [113]:
full_F = ip.cvx_opt_clusters(alarm_PDG, varname_clusters=[[V.name for V in alarm_PDG.varlist]], 
    also_idef=False,verbose=True).marginals[0]
full_F[smoke, lung, bronc]

[(0, ['asia', 'smoke', 'bronc', 'either', 'dysp', 'lung', 'tub', 'xray'])]
[1] adding constr  p38 :  1 -> asia
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [0]
	[1 / 2] components
	about to construct expcone
	mus[i].shape  (256,)
	expcone constructed
[2] adding constr  p39 :  smoke -> bronc
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1, 2]
	[3 / 4] components
	about to construct expcone
	mus[i].shape  (256,)
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1]
	[1 / 2] components
	expcone constructed
[3] adding constr  p40 :  either×bronc -> dysp
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 2, 4]
	[7 / 8] components
	about to construct expcone
	mus[i].shape  (256,)
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 2]
	[3 / 4] components
	expcone constructed
[4] adding constr  p41 :  tub×lung -> either
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [6, 5, 3]
	[7 / 8] components
	about to c

,"((yes, yes), yes)","((yes, yes), no)","((yes, no), yes)","((yes, no), no)","((no, yes), yes)","((no, yes), no)","((no, no), yes)","((no, no), no)"
⋆,0.029262,0.020738,0.270738,0.179262,0.002399,0.002601,0.147601,0.347399


In [112]:
full_T = ip.cvx_opt_clusters(alarm_PDG, varname_clusters=[[V.name for V in alarm_PDG.varlist]],
     also_idef=True, verbose=True).marginals[0]

[(0, ['asia', 'smoke', 'bronc', 'either', 'dysp', 'lung', 'tub', 'xray'])]
[1] adding constr  p38 :  1 -> asia
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [0]
	[1 / 2] components
	about to construct expcone
	mus[i].shape  (256,)
	expcone constructed
[2] adding constr  p39 :  smoke -> bronc
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1, 2]
	[3 / 4] components
	about to construct expcone
	mus[i].shape  (256,)
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1]
	[1 / 2] components
	expcone constructed
[3] adding constr  p40 :  either×bronc -> dysp
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 2, 4]
	[7 / 8] components
	about to construct expcone
	mus[i].shape  (256,)
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 2]
	[3 / 4] components
	expcone constructed
[4] adding constr  p41 :  tub×lung -> either
	about to marginalize
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [6, 5, 3]
	[7 / 8] components
	about to c

In [96]:
full_T[smoke,lung,bronc]

,"((yes, yes), yes)","((yes, yes), no)","((yes, no), yes)","((yes, no), no)","((no, yes), yes)","((no, yes), no)","((no, no), yes)","((no, no), no)"
⋆,0.03,0.02,0.27,0.18,0.0015,0.0035,0.1485,0.3465


In [123]:
jointF = ip.cvx_opt_joint(alarm_PDG, also_idef=False)
jointT = ip.cvx_opt_joint(alarm_PDG, also_idef=True)
marginalsT = ip.cvx_opt_clusters(alarm_PDG, also_idef=True, verbose=True).marginals
marginalsF = ip.cvx_opt_clusters(alarm_PDG, also_idef=True, verbose=False).marginals

	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [0]
	[1 / 2] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1, 2]
	[3 / 4] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1]
	[1 / 2] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 2, 4]
	[7 / 8] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 2]
	[3 / 4] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [6, 5, 3]
	[7 / 8] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [6, 5]
	[3 / 4] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1, 5]
	[3 / 4] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1]
	[1 / 2] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [1]
	[1 / 2] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [0, 6]
	[3 / 4] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [0]
	[1 / 2] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) [3, 7]
	[3 / 4] components
	marginalizing :  256 (2, 2, 2, 2, 2, 2, 2, 2) 

In [116]:
from pgmpy.inference.ExactInference import BeliefPropagation, VariableElimination

bp = BeliefPropagation(alarm)
ve = VariableElimination(alarm)

bp.calibrate()

# takes 48 seconds;
# ... or 1m 2sec
# takes 5GB RAM. 

In [7]:
del bp

In [26]:
[f.values.size for f in bp.get_clique_beliefs().values()]

[8, 32, 32, 16]

In [131]:
np.set_printoptions(precision=3)

vals = ve.query(list(alarm.nodes())).values


print('BP\t\t', bp.get_clique_beliefs()[('dysp', 'bronc', 'either')].values.reshape(-1))
print('var elim\t', ve.query(['dysp', 'bronc', 'either']).values.reshape(-1))
print()

print('cvx-cluster-F\t', marginalsF[0][dysp,bronc,either].to_numpy().reshape(-1))
print('cvx-cluster-T\t',marginalsT[0][dysp,bronc,either].to_numpy().reshape(-1))
print('cvx-big-clust-F\t', full_F[dysp,bronc,either].to_numpy().reshape(-1))
print('cvx-big-clust-T\t', full_T[dysp,bronc,either].to_numpy().reshape(-1))
print('cvx-joint-F\t', jointF[dysp,bronc,either].to_numpy().reshape(-1))
print('cvx-joint-T\t', jointT[dysp,bronc,either].to_numpy().reshape(-1))
print()


print('factor-product\t', alarm_PDG.factor_product()[dysp,bronc,either].to_numpy().reshape(-1))


# print(vals.shape, vals.sum())
rjd = RJD(vals, alarm_PDG.varlist)
print('var elim2\t', rjd[dysp,bronc,either].to_numpy().reshape(-1))


Finding Elimination Order: : 100%|██████████| 4/4 [01:22<00:00, 20.63s/it]
Finding Elimination Order: : : 0it [01:22, ?it/s]
0it [00:00, ?it/s]n Order: : : 0it [00:00, ?it/s]





Eliminating: tub: 100%|██████████| 4/4 [00:00<00:00, 216.04it/s]

BP		 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]
var elim	 [0.032 0.331 0.02  0.052 0.004 0.083 0.009 0.469]

cvx-cluster-F	 [0.032 0.29  0.023 0.052 0.004 0.124 0.006 0.469]
cvx-cluster-T	 [0.032 0.29  0.023 0.052 0.004 0.124 0.006 0.469]
cvx-big-clust-F	 [0.031 0.291 0.022 0.052 0.003 0.125 0.005 0.471]
cvx-big-clust-T	 [0.032 0.29  0.023 0.052 0.004 0.124 0.006 0.469]
cvx-joint-F	 [0.031 0.291 0.022 0.052 0.003 0.125 0.005 0.471]
cvx-joint-T	 [0.032 0.29  0.023 0.052 0.004 0.124 0.006 0.469]

factor-product	 [0.032 0.29  0.023 0.052 0.004 0.124 0.006 0.469]
(2, 2, 2, 2, 2, 2, 2, 2) 1.0
var elim2	 [2.500e-04 4.950e-03 4.750e-03 4.901e-01 2.500e-04 4.950e-03 4.750e-03
 4.900e-01]
